# CLASE 5 – Modelado Avanzado de Clases en POO

### Objetivos

* Diseñar soluciones empresariales utilizando modelado orientado a objetos.
* Modelar relaciones complejas entre clases: **composición y agregación**.
* Aplicar **herencia, abstracción, interfaces (simuladas)**.
* Implementar un sistema funcional con buenas prácticas de diseño.

---


## Relaciones avanzadas entre clases

### **Herencia (IS A / es un)**

Una clase hija adquiere atributos y métodos de su clase padre.

Ejemplo:
*Un Gerente es un Empleado.*

---



### **Composición (HAS A / “tiene un”) → No puede existir sin su componente**

Es una relación donde:
* Un objeto no puede existir sin el objeto que lo contiene.
* Una clase contiene a otra como parte esencial** de su estructura.
* Si el objeto principal deja de existir, **su componente también desaparece**.
* Relación 1:1 ó 1:N en diagramas E-R o Bases de datos

Ejemplo sencillo:

> Un *Restaurante* **tiene una** *Cocina*.
> Si el restaurante cierra, **la cocina ya no existe**.


Piensa en composición como “parte integral”.
Si se elimina el todo, se eliminan las partes.

---



### **Agregación (HAS A / “tiene un”) → Pueden existir por separado**

Es una relación entre clases donde:
* Los objetos pueden existir por separado.
* Una clase utiliza a otra, pero **no depende de ella para existir**.
* Relación 1:N opcional en diagramas E-R o Bases de datos

Ejemplo:

> Un *Restaurante* **tiene clientes**, pero si el restaurante cierra,
> **los clientes siguen existiendo**.


Piensa en agregación como una colaboración temporal, **no una dependencia total**.

---


### Diferencias resumidas

| Relación    | Dependencia                         | Ejemplo               |
| ----------- | ----------------------------------- | --------------------- |
| Herencia    | Fuerte (es un)                      | Gerente → Empleado    |
| Composición | Muy fuerte (tiene un ✔ obligatorio) | Restaurante → Cocina  |
| Agregación  | Baja (tiene un pero no depende)     | Restaurante → Cliente |

---


## Ejemplo práctico – Sistema Restaurante - Aplicando composición, agregación, herencia y clases abstractas



In [ ]:
from abc import ABC, abstractmethod   # Importamos ABC y abstractmethod para crear clases abstractas
from typing import List               # Importamos List para usar anotaciones de listas de objetos


# ===============================
# 1. Clase Abstracta Persona
# ===============================
class Persona(ABC):                   # Declaramos la clase Persona como abstracta (no se puede instanciar)
    """Superclase abstracta que representa una persona."""

    def __init__(self, nombre: str) -> None:
        # El constructor recibe el nombre y lo asigna usando el setter (para que pase por la validación)
        self.nombre = nombre          # Aquí NO se usa self._nombre directo, sino el setter 'nombre'

    @property
    def nombre(self) -> str:          # Getter del atributo 'nombre'
        """Obtiene el nombre de la persona."""
        # Devuelve el valor del atributo privado _nombre
        return self._nombre           # _nombre es el atributo "real" almacenado en el objeto

    @nombre.setter
    def nombre(self, nuevo_nombre: str) -> None:
        """Valida y asigna el nombre."""
        # Validamos que 'nuevo_nombre' sea una cadena (str) y que no esté vacía o llena de espacios
        if isinstance(nuevo_nombre, str) and nuevo_nombre.strip():   # strip() elimina espacios al inicio/fin
            self._nombre = nuevo_nombre                              # Si es válido, se guarda en _nombre
        else:
            # Si no cumple las condiciones, lanzamos una excepción para evitar estados inválidos
            raise ValueError("El nombre debe ser una cadena de texto válida.")

    @abstractmethod
    def presentar(self) -> None:      # Declaramos un método abstracto
        """Método obligatorio que deben implementar las subclases."""
        # Al ser abstracto, aquí no se implementa nada. Obliga a las clases hijas a definirlo.
        pass


# ===============================
# 2. Clases hijas
# ===============================
class Cliente(Persona):               # Cliente hereda de Persona (subclase concreta)
    def presentar(self) -> None:      # Implementa el método abstracto presentar()
        # Usa f-string para mostrar un mensaje usando el nombre del cliente
        print(f"Cliente {self.nombre} ha llegado al restaurante.")


class Empleado(Persona):              # Empleado también hereda de Persona
    # Esta clase sigue siendo abstracta, porque define un método abstracto adicional
    @abstractmethod
    def trabajar(self) -> None:       # Método que TODA subclase de Empleado debe implementar
        # No se implementa aquí; solo se declara como obligatorio
        pass


class Mesero(Empleado):               # Mesero es una subclase concreta de Empleado
    def presentar(self) -> None:      # Implementa cómo se presenta un Mesero
        print(f"Mesero {self.nombre} está listo para atender.")

    def trabajar(self) -> None:       # Implementa el método abstracto trabajar()
        print(f"{self.nombre} está tomando pedidos.")  # Simula la acción del mesero


class Chef(Empleado):                 # Chef también es una subclase concreta de Empleado
    def presentar(self) -> None:      # Implementa cómo se presenta un Chef
        print(f"Chef {self.nombre} está en la cocina.")

    def trabajar(self) -> None:       # Implementa el método abstracto trabajar()
        print(f"{self.nombre} está preparando los platos.")  # Simula la acción del chef


# ===============================
# 3. Composición (Cocina)
# ===============================
class Cocina:
    """Representa la cocina. Existe solo dentro del restaurante."""

    def __init__(self, chefs: List[Chef]) -> None:
        # El constructor recibe una lista de objetos Chef
        # Asignamos usando el setter para asegurarnos de que sean chefs válidos
        self.chefs = chefs            # Esto llama internamente a @chefs.setter

    @property
    def chefs(self) -> List[Chef]:     # Getter de la lista de chefs
        """Obtiene la lista de chefs."""
        # Devuelve la lista privada _chefs
        return self._chefs

    @chefs.setter
    def chefs(self, lista_chefs: List[Chef]) -> None:
        """Valida la lista de chefs."""
        # Validamos que 'lista_chefs' sea una lista y que todos sus elementos sean instancias de Chef
        if isinstance(lista_chefs, list) and all(isinstance(c, Chef) for c in lista_chefs):
            self._chefs = lista_chefs  # Si es válida, se asigna a _chefs
        else:
            # Lanzamos un error si no es una lista válida de chefs
            raise ValueError("Debe proporcionar una lista válida de objetos de tipo Chef.")

    def operar(self) -> None:
        """Ejecuta el trabajo de todos los chefs."""
        # Recorre cada chef en la lista y le ordena trabajar
        for chef in self.chefs:       # self.chefs usa el getter
            chef.trabajar()           # Llama al método trabajar() de cada objeto Chef


# ===============================
# 4. Agregación (Restaurante)
# ===============================
class Restaurante:
    def __init__(self, nombre: str) -> None:
        # Asigna el nombre del restaurante usando el setter (con validación)
        self.nombre = nombre          # Llama a @nombre.setter
        # Inicializa la lista de clientes (agregación: los clientes pueden existir fuera del restaurante)
        self.clientes: List[Cliente] = []
        # Crea un objeto Cocina que pertenece al restaurante (composición fuerte)
        # Aquí se instancian dos objetos Chef y se pasan a Cocina
        self.cocina = Cocina([Chef("Carlos"), Chef("Ana")])

    @property
    def nombre(self) -> str:
        # Devuelve el nombre privado del restaurante
        return self._nombre

    @nombre.setter
    def nombre(self, nuevo_nombre: str) -> None:
        """Valida el nombre del restaurante."""
        # Verifica que sea una cadena de texto no vacía
        if isinstance(nuevo_nombre, str) and nuevo_nombre.strip():
            self._nombre = nuevo_nombre  # Asigna el nombre si es válido
        else:
            # Si no, lanza una excepción para evitar un objeto con nombre inválido
            raise ValueError("El nombre del restaurante debe ser un texto válido.")

    def agregar_cliente(self, cliente: Cliente) -> None:
        """Agregación: los clientes pueden existir fuera del restaurante."""
        # Validamos que el objeto que entra sea de tipo Cliente
        if isinstance(cliente, Cliente):
            # Si es válido, lo agregamos a la lista de clientes del restaurante
            self.clientes.append(cliente)
        else:
            # Si no es un Cliente, se lanza un error
            raise ValueError("Solo se pueden agregar objetos de tipo Cliente.")

    def iniciar_servicio(self) -> None:
        """Simula el inicio del servicio del restaurante."""
        # Muestra un encabezado indicando que el restaurante comienza su servicio
        print(f"\n=== Restaurante {self.nombre} iniciando servicio ===")

        print("\nAtendiendo clientes:")
        # Recorre todos los clientes agregados al restaurante
        for cliente in self.clientes:
            # Llama al método presentar() de cada cliente
            # Polimorfismo: si hubiera subclases de Cliente, se usaría la versión correspondiente
            cliente.presentar()

        print("\nCocina en operación:")
        # Llama al método operar() de la cocina, que hace que todos los chefs trabajen
        self.cocina.operar()


# ===============================
# MAIN
# ===============================
def main() -> None:
    # Crea una instancia de Restaurante con el nombre "La Buena Mesa"
    restaurante = Restaurante("La Buena Mesa")

    # Agrega un cliente llamado "Luis" al restaurante (agregación)
    restaurante.agregar_cliente(Cliente("Luis"))
    # Agrega un segundo cliente llamado "María"
    restaurante.agregar_cliente(Cliente("María"))

    # Línea de prueba opcional: generar un error si el nombre está vacío
    # restaurante.agregar_cliente(Cliente(""))  # Esto activaría la validación del nombre

    # Llama al método que simula el inicio de las operaciones del restaurante
    restaurante.iniciar_servicio()


# Punto de entrada del programa
if __name__ == "__main__":
    # Si el archivo se ejecuta directamente (no importado como módulo), se llama a main()
    main()
